# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

[2026-01-23 03:27:14] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2026-01-23 03:27:14] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2026-01-23 03:27:14] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2026-01-23 03:27:14] INFO lfm2.py:102: Registered SGLang Lfm2Config to override HuggingFace's version


[2026-01-23 03:27:17] INFO server_args.py:1724: Attention backend not specified. Use fa3 backend by default.


...

Capturing batches (bs=88 avail_mem=13.25 GB):  20%|██        | 4/20 [00:01<00:03,  4.99it/s]

Capturing batches (bs=40 avail_mem=13.21 GB):  50%|█████     | 10/20 [00:01<00:00, 12.69it/s]

Capturing batches (bs=16 avail_mem=13.20 GB):  65%|██████▌   | 13/20 [00:01<00:00, 15.96it/s]

Capturing batches (bs=4 avail_mem=13.19 GB):  80%|████████  | 16/20 [00:01<00:00, 12.93it/s]

Capturing batches (bs=1 avail_mem=13.18 GB): 100%|██████████| 20/20 [00:01<00:00, 10.49it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Anna. I'm 22 years old and I'm from the countryside. I got into computer science because I was interested in it. I think that computers are really useful and change the way we live. I want to become a computer scientist. What do you think? Do you think computers will become even more useful in the future? What do you think of your generation? What do you think of the world? I like to write, and I'm a little writer. What do you think about creativity? Do you think that we will be creative in the future? What do you think of teachers and how they teach? How do
Prompt: The president of the United States is
Generated text:  seeking to choose the next in line for the presidency. The candidate must be at least 35 years old and have lived in the United States for at least 10 years. Additionally, the candidate must have at least 200,000 votes cast for them in a particular election. The candidate is to be voted on by the American people. 

If the presi

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name], and I'm a [job title] at [company name]. I'm excited to meet you and learn more about you. What can you tell me about yourself? [Name] is a [job title] at [company name]. I'm excited to meet you and learn more about you. What can you tell me about yourself? [Name] is a [job title] at [company name]. I'm excited to meet you and learn more about you. What can you tell me about yourself? [Name] is a [job title] at [company name]. I'm excited to meet you and learn more about

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris, the city known for its iconic landmarks such as the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum. It is also home to the French Parliament, the French National Museum, and the French Parliament building. Paris is a bustling metropolis with a rich history and culture, and is a popular tourist destination. It is also a major financial center and a major hub for international business. The city is known for its fashion industry, and is home to many famous fashion houses and boutiques. Paris is a city that is constantly evolving and changing, with new developments and cultural events taking place throughout the year. The

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  likely to be characterized by rapid advancements in areas such as machine learning, natural language processing, and computer vision. These technologies are expected to continue to evolve and improve, leading to more sophisticated and accurate AI systems that can perform a wide range of tasks with increasing accuracy and efficiency. Some potential future trends in AI include:

1. Increased integration with human intelligence: As AI systems become more sophisticated, they are likely to become more integrated with human intelligence, allowing them to learn from and adapt to the behavior and preferences of humans.

2. Greater use of AI in healthcare: AI is already being used in healthcare to improve patient outcomes, reduce costs,



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  [Name], and I'm a [Age], [Gender] [Gender Identity] [Occupation]. I'm currently [Current Position or Project] and I'm passionate about [Your Passion]. I enjoy [Your Hobby or Interests] and I'm always looking for new experiences and adventures. I believe in the power of [Your Core Belief or Principle], and I'm always striving to learn and grow. I have a [Number of] [Experience] years of experience in [Your Field of Expertise], and I'm always looking for ways to improve myself and expand my knowledge. I'm confident in myself and am always striving

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris, a historical and cultural city known for its iconic landmarks and museums, including the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum. The city's importance dates back to ancient times, with its role 

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 [

Name

]

 and

...

 integrated

 into

 human

 society

,

In [6]:
llm.shutdown()